In [65]:
from astropy.io import fits
import os
import subprocess
import numpy as np

In [18]:
imageDirectory = '/data/mj1e16/kepler/cal/'
dirlist = os.listdir(imageDirectory)
dirlist = [x for x in dirlist if x[0]!='.']
dirlist = [imageDirectory+x for x in dirlist]


In [10]:
supernovaStart = [56176.666,55846.320,55928.414]

In [26]:
def findClosestImage(supernovaStart,dirlist):
    answer = []
    for y in range(len(supernovaStart)):
        timediff = []
        for x in range(len(dirlist)):
            #print(dirlist[x])
            hdu = fits.open(dirlist[x])
            header = hdu[1].header
            timediff.append(abs(header['MJDSTART']-supernovaStart[y]))

        closest = min(timediff)
        imageIndex =  timediff.index(closest)   
        answer.append({'timediff':closest,'index':imageIndex,'image':dirlist[imageIndex]})
    return answer

In [92]:
print(answer)

[{'index': 46, 'timediff': 7.854872060001071, 'image': '/data/mj1e16/kepler/cal/kplr2012242195726_ffi-cal.fits'}, {'index': 6, 'timediff': 13.539371960003336, 'image': '/data/mj1e16/kepler/cal/kplr2011271191331_ffi-cal.fits'}, {'index': 41, 'timediff': 2.427519040000334, 'image': '/data/mj1e16/kepler/cal/kplr2012004204112_ffi-cal.fits'}]


In [34]:
answer = findClosestImage(supernovaStart,dirlist)

In [66]:
def correctExtensionOrder(imageName):
    hdu_list = fits.open(imageName)
    extensionOrder = np.zeros(len(hdu_list))
    for channel in range(1,len(hdu_list)):
        hdr = hdu_list[channel].header
        location = hdr['SKYGROUP'] # independant location
        extensionOrder[location] = channel # ordered from 0-84 channel with extension in place of name
    return extensionOrder

In [67]:
extensionOrders = []
for image in dirlist:
    extensionOrders.append(correctExtensionOrder(image))

In [61]:
def makeDS9RegFile(xcoords,ycoords,fileNameBase):
    
    for x in range(len(xcoords)):
        bigString = 'global color=lightgreen\nWCS\n'
        bigString += 'circle({},{},0.1)\n'.format(xcoords[x],ycoords[x])
        fileName = fileNameBase+str(x)+'.reg'
        with open(fileName,'w') as f:
            f.write(bigString)

In [78]:
names = ['KSN 2012a','KSN 2011b','KSN 2011c'] 
# ccds = [61,42,29]
# ccds = [37,41,81]
#ccds = [21,44,63]
ccds = [45,1,44]
ra = [293.3754458333333, 290.15641666666664, 291.1920374999999]
dec = [45.25035833333333, 38.252316666666665, 43.68074444444444]

In [62]:
makeDS9RegFile(ra,dec,'supernovae')

In [43]:
answer[0]['image']+str(ccds[0])

'/data/mj1e16/kepler/cal/kplr2012242195726_ffi-cal.fits61'

In [76]:
for x in range(len(names)):
    #extension = int(extensionOrders[answer[x]['index']][ccds[x]])
    extension = ccds[x]
    print(extension)
    print('ds9 '+answer[x]['image']+'[{}]'.format(extension)+' -regions supernovae{}.reg'.format(x))
    #subprocess.call(['ds9',answer[x]['image']+'[{}]'.format(ccds[x])]) #,'-regions','supernovae{}.reg'.format(x)]

45
ds9 /data/mj1e16/kepler/cal/kplr2012242195726_ffi-cal.fits[45] -regions supernovae0.reg
1
ds9 /data/mj1e16/kepler/cal/kplr2011271191331_ffi-cal.fits[1] -regions supernovae1.reg
43
ds9 /data/mj1e16/kepler/cal/kplr2012004204112_ffi-cal.fits[43] -regions supernovae2.reg


In [ ]:
# ds9 /data/mj1e16/kepler/cal/kplr2012242195726_ffi-cal.fits[45] -regions supernovae0.reg
# ds9 /data/mj1e16/kepler/cal/kplr2011271191331_ffi-cal.fits[1] -regions supernovae1.reg
# ds9 /data/mj1e16/kepler/cal/kplr2012004204112_ffi-cal.fits[44] -regions supernovae2.reg

In [80]:
def hotpants(imNo,channel,extensionOrders,imagelist,name,medianDir='/data/mj1e16/kepler/medians/'):
    ccdChannel = int(extensionOrders[imNo][channel])
    inim = imagelist[imNo]+'[{}]'.format(ccdChannel)
    tmplim = medianDir+'meidian_{}.fits'.format(channel)
    print(tmplim)
#     outim = '/home/mj1e16/testImages/supernovaDiff_'+name+'.fits'
#     # outim = '/data/mj1e16/kepler/properDiff/diff_imNo{}_channel{}.fits'.format(imNo,channel)
#     subprocess.call(['./hotpants','-inim',inim,'-tmplim',tmplim,'-outim',outim,'-v','0'])

In [81]:
for x in range(len(ccds)):
    hotpants(answer[x]['index'],ccds[x],extensionOrders,dirlist,names[x])

/data/mj1e16/kepler/medians/meidian_45.fits
/data/mj1e16/kepler/medians/meidian_1.fits
/data/mj1e16/kepler/medians/meidian_44.fits


In [97]:
medians = [37,81,41]
os.chdir('/data/mj1e16/kepler/hotpants-master/')
for x in range(len(medians)):
    inim = dirlist[answer[x]['index']]+'[{}]'.format(ccds[x])
    tmplim = '/data/mj1e16/kepler/medians/meidian_{}.fits'.format(medians[x])
    outim = '/home/mj1e16/testImages/supernovae_{}.fits'.format(x)
    #print(outim)
    print('./hotpants '+'-inim '+inim+' -tmplim '+tmplim+' -outim '+outim)
    #subprocess.call(['./hotpants','-inim',inim,'-tmplim',tmplim,'-outim',outim])

./hotpants -inim /data/mj1e16/kepler/cal/kplr2012242195726_ffi-cal.fits[45] -tmplim /data/mj1e16/kepler/medians/meidian_37.fits -outim /home/mj1e16/testImages/supernovae_0.fits
./hotpants -inim /data/mj1e16/kepler/cal/kplr2011271191331_ffi-cal.fits[1] -tmplim /data/mj1e16/kepler/medians/meidian_81.fits -outim /home/mj1e16/testImages/supernovae_1.fits
./hotpants -inim /data/mj1e16/kepler/cal/kplr2012004204112_ffi-cal.fits[44] -tmplim /data/mj1e16/kepler/medians/meidian_41.fits -outim /home/mj1e16/testImages/supernovae_2.fits


In [93]:
dirlist

['/data/mj1e16/kepler/cal/kplr2012121122500_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2011240181752_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2009322233047_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2010111125026_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2011177110110_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2013011160902_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2011271191331_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2010174164113_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2012088132324_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2010265195356_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2009170043915_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2011303191211_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2011334181008_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2009260000800_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2009114204835_ffi-cal.fits',
 '/data/mj1e16/kepler/cal/kplr2012341215621_ffi-cal.fits